<a href="https://colab.research.google.com/github/coreygoavec/Collection-of-Works/blob/main/Goavec%2C_Corey_DS_4220_HW_%5B_4%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Load in the data
cancer_data <- read.csv("/content/sample_data/breast cancer.csv")
# head(cancer_data)

QUESTION 1

In [ ]:
# Building models to predict whether the tumor is malignant ("M") or benign ("B") using all the classification models.

# Install packages
# install.packages(c("MASS", "e1071"))
library("MASS")
library("e1071")

# Converting dependent variable, "diagnosis", to be a factor.
cancer_data$diagnosis <- as.factor(cancer_data$diagnosis)

# Converting dependent variable, "diagnosis", to be numeric.
# cancer_data$diagnosis <- ifelse(cancer_data$diagnosis == "M", 1, 0)

# Naive Bayes, Logistic Regression, and Linear Discriminant Analysis models
nb_model <- naiveBayes(diagnosis ~., data = cancer_data, type = 'class')
lr_model <- glm(diagnosis ~., data = cancer_data, family = 'binomial')
lda_model <- lda(diagnosis ~., data = cancer_data)

# # Getting predictions for the models

# # NB model prediction
nb_pred <- predict(nb_model, newdata = cancer_data, type = 'class')

# # LR model prediction
lr_pred <- predict(lr_model, type = 'response')

# # LDA model prediction
lda_pred <- predict(lda_model, newdata = cancer_data)

# # Getting class labels for LR model
lr_class <- ifelse(lr_pred > 0.5, "M", "B")

# # Getting class labels for LDA model
lda_prob <- as.data.frame(lda_pred$posterior)
lda_prob$class <- ifelse(lda_prob$B > 0.5, "M", "B") #Gives us the class but data is now focused on Benign tumors

Warning message:
“glm.fit: algorithm did not converge”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”


QUESTION 2

In [ ]:
# Finding the probability that a tumor is benign or malignant.
# Probabilities along with summary statistics

# install.packages("caret")
library("caret")

# NB probablities
nb_mod_prob <- naiveBayes(diagnosis ~ ., data = cancer_data, type = 'raw')
nb_prob <- as.data.frame(predict(nb_mod_prob, newdata = cancer_data, type = 'raw'))
summary(nb_prob)

       B                M         
 Min.   :0.0000   Min.   :0.0000  
 1st Qu.:0.0000   1st Qu.:0.0000  
 Median :1.0000   Median :0.0000  
 Mean   :0.6396   Mean   :0.3604  
 3rd Qu.:1.0000   3rd Qu.:1.0000  
 Max.   :1.0000   Max.   :1.0000  

In [ ]:
# LDA probabilities
summary(lda_prob$B)

    Min.  1st Qu.   Median     Mean  3rd Qu.     Max. 
0.000000 0.009607 0.994359 0.656730 0.999795 1.000000 

In [ ]:
# LR probabilites
lr_prob <- 1/(1 + exp(-lr_pred))
summary(lr_prob)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.5000  0.5000  0.5000  0.5861  0.7311  0.7311 

In [ ]:
# On average the probability that a tumor being Benign is 63.96% for Naive Bayes' model,
# 65.67% for the LDA model, and 58.61% for the LR model.

# A.
# There is a difference in the two groups, and the probabilities
# agree with one another that the chances of a tumor being Benign are higher than
# a tumor being Malignant.

In [ ]:
# B.
# Using a statistical test to conclude if there is difference.

# install.packages("FSA")
# install.packages("nortest")
library("FSA")
library("nortest")

# AD test for tumors that are Benign(B) across each tumor.
benign_test <- cancer_data[cancer_data$diagnosis == "B", ]
ad.test(benign_test$id)

# AS test for tumors that are Malignant(M) across each tumor.
malignant_test <- cancer_data[cancer_data$diagnosis == "M", ]
ad.test(malignant_test$id)

# Identify p-value, if less than 0.05, reject null hypothesis and accept alternative hypothesis, being
# the data does not follow a normal distribution.

# P-values from the AD tests indicate we have p-values below 0.05 and should therefore accept the alternative hypothesis

# Since normality can not be assumed here, perform a KW test to see if the three models are significant.
# Conducting KW test
kruskal.test(diagnosis ~ id, data = cancer_data)


	Anderson-Darling normality test

data:  benign_test$id
A = 102.32, p-value < 2.2e-16



	Anderson-Darling normality test

data:  malignant_test$id
A = 57.024, p-value < 2.2e-16



	Kruskal-Wallis rank sum test

data:  diagnosis by id
Kruskal-Wallis chi-squared = 568, df = 568, p-value = 0.4921


In [ ]:
# From the KW test we get a p-value of 0.4921 which is not statistically significant, meaning there is not enough
# evidence to reject the null hypothesis and instead accept it, meaning that the means of the three models
# are similar to one another.

QUESTION 3

In [ ]:
# Picking two numeric variables and interpreting their coefficients.
# LR model coefficients
print(lr_model)


Call:  glm(formula = diagnosis ~ ., family = "binomial", data = cancer_data)

Coefficients:
            (Intercept)                       id              radius_mean  
              9.535e+01               -8.422e-07               -1.856e+03  
           texture_mean           perimeter_mean                area_mean  
              2.613e+01               -5.121e+00                1.850e+01  
        smoothness_mean         compactness_mean           concavity_mean  
              2.267e+04               -1.781e+04                7.035e+03  
    concave.points_mean            symmetry_mean   fractal_dimension_mean  
              1.449e+04               -7.373e+03                1.617e+04  
              radius_se               texture_se             perimeter_se  
             -7.733e+02               -2.418e+02               -3.304e+02  
                area_se            smoothness_se           compactness_se  
              5.952e+01                2.557e+03                3.204e+

In [ ]:
# Interpreting numeric variable 'perimeter_mean'
perimeter_mean <- (-5.121e+00)
print(perimeter_mean)

# Getting the odds for 'perimeter_mean'
print(exp(perimeter_mean))

# The odds that a tumors perimeter mean gets larger, increases by a factor of 1.00597005 for each tumor measured.

[1] -5.121
[1] 0.00597005


In [ ]:
# Interpreting numeric variable 'area_mean'
area_mean <- (-1.850e+01)
print(area_mean)

# Getting the odds for 'radius_mean'
print(exp(area_mean))

# The odds of a tumors area mean gets larger, increases by a factor of 1.00000000923745 for each tumor measured.

[1] -18.5
[1] 9.23745e-09
